In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
from pyspark.sql.functions import *

a) Create Airline Pysaprk sql dataFrame

In [0]:
flights_delay_df = spark.read.csv("dbfs:/FileStore/tables/Flights_Delay.csv", header = True, inferSchema = True)
type(flights_delay_df)

Out[3]: pyspark.sql.dataframe.DataFrame

b) Describe the table schema & show 10 rows of Dataset

In [0]:
flights_delay_df.printSchema()
flights_delay_df.show(10)

root
 |-- ID: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true

c) Average arrival delay caused by airlines.

In [0]:
flights_delay_df.select(mean("ARRIVAL_DELAY")).show()

+------------------+
|avg(ARRIVAL_DELAY)|
+------------------+
| 7.545457931394093|
+------------------+



d) DAY_OF_MONTH with average Arrival delays.

In [0]:
flights_delay_df.createOrReplaceTempView("flightData")
spark.sql("select DAY, AVG(ARRIVAL_DELAY) as avgDelay from flightData group by DAY order by avgDelay desc").show()

+---+------------------+
|DAY|          avgDelay|
+---+------------------+
|  3|18.141541038525965|
|  4|17.157790927021697|
|  5| 16.23861262014208|
|  2|15.046014790468364|
|  1|14.807807807807809|
| 26| 11.96778269109286|
| 12| 11.24892703862661|
|  6|10.608832807570979|
| 16| 9.124321062160531|
| 17| 8.761435608726249|
| 21| 6.919860627177701|
| 22| 6.550920245398773|
| 24| 5.737543859649123|
|  8| 5.232349165596919|
| 25| 4.903708523096942|
| 27| 4.706711409395973|
| 30| 4.471478463329452|
|  9| 4.421887390959556|
| 23| 4.207086133170434|
| 11|3.9912935323383083|
+---+------------------+
only showing top 20 rows



e) Days of months with median Departure delays.

In [0]:
spark.sql("select DAY, percentile_approx(DEPARTURE_DELAY,0.5) as median from flightData group by DAY order by median").show()

+---+------+
|DAY|median|
+---+------+
| 30|    -3|
| 29|    -3|
| 31|    -3|
| 20|    -2|
| 21|    -2|
| 19|    -2|
| 27|    -2|
| 24|    -2|
| 11|    -2|
| 14|    -2|
| 18|    -2|
|  7|    -2|
| 10|    -2|
| 28|    -2|
| 25|    -2|
| 26|    -1|
| 22|    -1|
| 13|    -1|
| 15|    -1|
| 16|    -1|
+---+------+
only showing top 20 rows



f) Arrange weekdays with respect to the average arrival delays.

In [0]:
spark.sql("select DAY_OF_WEEK, avg(ARRIVAL_DELAY) as avg_arr_delay from flightData group by DAY_OF_WEEK order by avg_arr_delay ").show()

+-----------+------------------+
|DAY_OF_WEEK|     avg_arr_delay|
+-----------+------------------+
|          6| 4.888689138576779|
|          3| 5.587079407806191|
|          5| 6.010538373424971|
|          4| 7.174969021065675|
|          2| 8.033644102148358|
|          7|10.110840438489646|
|          1|10.807447207297264|
+-----------+------------------+



g) Show Analysis for each month with total number of cancellations. (Descending).

In [0]:
spark.sql("select MONTH, count(CANCELLED) as total_cancellation from flightData group by MONTH order by total_cancellation desc ").show()

+-----+------------------+
|MONTH|total_cancellation|
+-----+------------------+
|    1|             25064|
|    2|             22936|
|    3|              8000|
+-----+------------------+



h) Show the top 10 busiest airport. [Hint: Sum Total of Arrival and Departures from an Airport]

In [0]:
spark.sql("select ORIGIN_AIRPORT as Aiports, count(ORIGIN_AIRPORT)+count(DESTINATION_AIRPORT) as Total from flightData group by ORIGIN_AIRPORT order by Total DESC").show(10)


+-------+-----+
|Aiports|Total|
+-------+-----+
|    ATL| 7154|
|    ORD| 5792|
|    DFW| 5602|
|    LAX| 4094|
|    DEN| 4070|
|    IAH| 3224|
|    PHX| 3178|
|    SFO| 3010|
|    LAS| 2832|
|    MCO| 2400|
+-------+-----+
only showing top 10 rows



i) Show the airlines that make the maximum number of cancellations.

In [0]:
spark.sql("select AIRLINE, count(CANCELLED) from flightData group by AIRLINE limit 1").show()

+-------+----------------+
|AIRLINE|count(CANCELLED)|
+-------+----------------+
|     UA|            4701|
+-------+----------------+



j) Find and order airlines in descending that make the most number of diversions.

In [0]:
spark.sql("select AIRLINE DIVERTED, count(DIVERTED) from flightData group by AIRLINE order by DIVERTED desc").show()

+--------+---------------+
|DIVERTED|count(DIVERTED)|
+--------+---------------+
|      WN|          11738|
|      VX|            573|
|      US|           3925|
|      UA|           4701|
|      OO|           5708|
|      NK|           1048|
|      MQ|           3502|
|      HA|            722|
|      F9|            794|
|      EV|           5916|
|      DL|           7989|
|      B6|           2548|
|      AS|           1586|
|      AA|           5250|
+--------+---------------+



k) Show days of month that seen the most number of diversion.

In [0]:
spark.sql("select DAY, count(DIVERTED) as Total_Diversion from flightData group by DAY order by Total_Diversion desc").show(5)

+---+---------------+
|DAY|Total_Diversion|
+---+---------------+
|  4|           2660|
|  2|           2644|
|  9|           2620|
|  5|           2619|
|  6|           2596|
+---+---------------+
only showing top 5 rows



l) Calculate mean and standard deviation of departure delay for all flights in minutes.

In [0]:
spark.sql("select DAY, mean(DEPARTURE_DELAY) as mean_dep_delay,stddev(DEPARTURE_DELAY) as std_dep_delay from flightData group by DAY order by DAY ").show()


+---+------------------+------------------+
|DAY|    mean_dep_delay|     std_dep_delay|
+---+------------------+------------------+
|  1|17.673439048562933| 57.38881728349448|
|  2|17.564667482660138| 42.99898669999238|
|  3| 18.36905754795663| 43.40352319391342|
|  4|18.744313725490198|  48.1419589553974|
|  5| 18.66306605733278|45.941946030389545|
|  6| 15.22995283018868| 42.96222118229427|
|  7| 8.346119536128457|  35.4521216264686|
|  8| 10.35820895522388| 38.38325627117254|
|  9| 9.273086029992108|33.218076313158186|
| 10| 5.046472564389697| 28.16670206462387|
| 11|  8.59764414135152| 35.01477772967999|
| 12|14.503056234718827| 40.69155240509875|
| 13| 7.863661971830986| 32.56170108868265|
| 14| 5.783637592968222|29.831266167509547|
| 15|        7.43359375|  29.6202372338009|
| 16|12.037860576923077|34.095252434834684|
| 17|13.855641205325858| 45.26104943695904|
| 18| 7.600970285021225|  32.6245693202197|
| 19| 6.648632218844985|24.968141615982823|
| 20| 8.375148632580261| 31.8922

m) Calculate mean and standard deviation of arrival delay for all flights in minutes.

In [0]:
spark.sql("select DAY, mean(ARRIVAL_DELAY) as mean_arr_delay,stddev(ARRIVAL_DELAY) as std_arr_delay from flightData group by DAY order by DAY ").show()


+---+--------------------+------------------+
|DAY|      mean_arr_delay|     std_arr_delay|
+---+--------------------+------------------+
|  1|  14.807807807807809|60.162466560747916|
|  2|  15.046014790468364| 45.41053926324797|
|  3|  18.141541038525965| 47.62733317283967|
|  4|  17.157790927021697|  50.6867084804893|
|  5|   16.23861262014208|49.015477523638985|
|  6|  10.608832807570979| 45.97548425759064|
|  7|  2.8309417040358746|  38.0841036714152|
|  8|   5.232349165596919| 42.19620894077951|
|  9|   4.421887390959556| 35.16249529954618|
| 10|-0.04705882352941...| 30.10448962258884|
| 11|  3.9912935323383083|37.444685427657966|
| 12|   11.24892703862661| 43.13283047516089|
| 13|  3.3769751693002257| 34.73947712329677|
| 14|  1.3299319727891157|32.464160298328366|
| 15|   2.966753585397653| 32.40494458288732|
| 16|   9.124321062160531| 37.89966086120615|
| 17|   8.761435608726249|  42.4139736363821|
| 18|  3.5693430656934306|34.163342329963804|
| 19|  1.6344282238442822|27.42884

n) Find all diverted Route from a source to destination Airport & which route is the most diverted

In [0]:
spark.sql("select ORIGIN_AIRPORT,DESTINATION_AIRPORT, count(DIVERTED) as Total_Diversion from flightData group by ORIGIN_AIRPORT,DESTINATION_AIRPORT order by Total_Diversion desc").show(5)


+--------------+-------------------+---------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|Total_Diversion|
+--------------+-------------------+---------------+
|           LAX|                JFK|            146|
|           LAX|                SFO|            142|
|           JFK|                LAX|            137|
|           SFO|                LAX|            131|
|           LAS|                LAX|            123|
+--------------+-------------------+---------------+
only showing top 5 rows



o) Finding AIRLINES with its total flight count, total number of flights arrival delayed by more than 30 Minutes, % of such flights delayed by more than 30 minutes when it is not Weekends with minimum count of flights from Airlines by more than 10. Also Exclude some of Airlines 'AK', 'HI', 'PR', 'VI' and arrange output in descending order by % of such count of flights.

In [0]:
# not getting